### __Survey__

In [1]:
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [ ]:
# Data of all states
folder_path = "../google-map-data/store-meta"

# List to hold all the state dfs
state_dataframes = []

# Loop through each file in the folder and append the df to the list
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df_state = pd.read_csv(file_path)
        state_dataframes.append(df_state)

# Combine all state DataFrames into one DataFrame
df_states = pd.concat(state_dataframes, ignore_index=True)
df_states

In [ ]:
# Dropping duplicate rows
df_states = df_states.drop_duplicates()
df_states

In [ ]:
# Keep only useful columns for df_states
columns_to_keep_states = ['gmap_id', 'latitude', 'longitude']
df_states = df_states[columns_to_keep_states]
df_states

In [ ]:
# Load the shapefile for states
states_gdf = gpd.read_file("../socio-economic-data/state shapefile/cb_2018_us_state_5m.shp")

# Ensure the CRS of states_gdf is EPSG:4326 for consistency
states_gdf = states_gdf.to_crs(epsg=4326)

# Convert df_states to a GeoDataFrame by creating a geometry column from latitude and longitude
df_states = df_states.copy()  # Make a copy to avoid modifying the original DataFrame
df_states['geometry'] = df_states.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
df_states_gdf = gpd.GeoDataFrame(df_states, geometry='geometry', crs="EPSG:4326")

# Ensure consistent data types for merging
df_states_gdf['gmap_id'] = df_states_gdf['gmap_id'].astype(str)

# Perform spatial intersection using overlay
df_states_with_state = gpd.overlay(df_states_gdf, states_gdf, how='intersection')

# Assign state names and keep only the relevant columns
df_states_with_state['state'] = df_states_with_state['NAME']
df_states = df_states_with_state[['gmap_id', 'latitude', 'longitude', 'state']]

# Final output
df_states

In [ ]:
# Load the review (health resource-related) data
df_review = pd.read_csv('../filtered-labeled-data/classed_review_2018.csv')
df_review

In [ ]:
# Keeping only the specified columns
columns_review = ['gmap_id', 'text', 'time', 'class']
df_review = df_review[columns_review]
df_review

In [ ]:
# Dropping duplicate rows from df_reviews
df_review = df_review.drop_duplicates()
df_review

In [ ]:
# Merge both dfs based on 'gmap_id' column
df_merged = pd.merge(df_states, df_review, on='gmap_id', how='inner')
df_merged

In [ ]:
# Convert the time column to datetime format
df_merged['time'] = pd.to_datetime(df_merged['time'])

# Function to calculate the review count and average sentiment for each state, per month
def calculate_monthly_sentiment(df):
    # Extract year and month from the time column
    df['month'] = df['time'].dt.to_period('M')

    # Group by state and year_month, calculate review count and sentiment
    grouped_df = df.groupby(['state', 'month']).agg(
        review_count=('gmap_id', 'size'),  # count the number of reviews
        total_sentiment=('class', lambda x: (x == 1).sum() - (x == -1).sum())  # calculate the total sentiment
    ).reset_index()

    # Calculate the average sentiment
    grouped_df['average_sentiment'] = grouped_df['total_sentiment'] / grouped_df['review_count']

    # Filter out months with less than 10 reviews
    filtered_df = grouped_df[grouped_df['review_count'] > 10]

    return filtered_df

# Calculate the monthly sentiment data
result_df = calculate_monthly_sentiment(df_merged)

# Display the final df
result_df

In [ ]:
# Now proceed with filtering as usual
result_df = result_df[(result_df['month'] >= '2020-04') & (result_df['month'] <= '2021-05')]
result_df

In [ ]:
# Save the 'filtered_county' as a CSV file
file_path = '../result- regression-data/sentiment_validation.csv'
result_df.to_csv(file_path, index=False)